# Create 3d clone of the scene

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# opens up the florr_plan image and lets you select 4 points on the floor which will be used to create a 3d volume
def create_volume(floor_plan, height):

    points = []
            
    def select_points_plan(event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            points.append([x, y, 0])
            print("floor plan point selected:", [x, y, 0])

    cv2.namedWindow("floor_plan")
    cv2.setMouseCallback("floor_plan", select_points_plan)

    while True:
        cv2.imshow("floor_plan", floor_plan)
        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            break

    cv2.destroyAllWindows()

    # copy points and add the height to the points
    original_matrix = np.array(points)
    modified_matrix = np.copy(original_matrix)
    modified_matrix[:,-1] = height

    return np.concatenate((original_matrix, modified_matrix), axis=0)

In [3]:
def create_image_association():

    image = cv2.imread("Ch4_image.png")

    points = []
            
    def select_points_plan(event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            points.append([x, y])
            print("floor plan point selected:", [x, y])

    cv2.namedWindow("image")
    cv2.setMouseCallback("image", select_points_plan)

    while True:
        cv2.imshow("image", image)
        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            break

    cv2.destroyAllWindows()

    # copy points and add the height to the points
    return np.array(points)

In [7]:
floor_plan = cv2.imread("Ch4_floor_plan.png")
points = [create_volume(floor_plan, 120) for vol in range(2)]

floor plan point selected: [463, 925, 0]
floor plan point selected: [412, 925, 0]
floor plan point selected: [412, 599, 0]
floor plan point selected: [338, 919, 0]
floor plan point selected: [291, 923, 0]


In [8]:
import plotly.graph_objs as go

# create a layout
layout = go.Layout(scene=dict(xaxis=dict(range=[650, 0], autorange=False),
                              yaxis=dict(range=[0, 1046], autorange=False),
                              zaxis=dict(range=[0, 650], autorange=False),
                              aspectmode='manual',
                              aspectratio=dict(x=1, y=1, z=1)))

# create a scatter plot
red_dot = points[0][0]
plots = [go.Scatter3d(x=vol[:,0], y=vol[:,1], z=vol[:,2], mode='lines', marker=dict(size=2)) for vol in points]
plots.append(go.Scatter3d(x=[red_dot[0]], y=[red_dot[1]], z=[red_dot[2]], mode='markers', marker=dict(size=5, color='red')))

# create a figure
fig = go.Figure(data=plots, layout=layout)

# show the figure
fig.show()

In [9]:
object_points_pre = np.concatenate(points)
object_points_pre

array([[463, 925,   0],
       [412, 925,   0],
       [412, 599,   0],
       [463, 925, 120],
       [412, 925, 120],
       [412, 599, 120],
       [338, 919,   0],
       [291, 923,   0],
       [338, 919, 120],
       [291, 923, 120]])

In [10]:
image_points_pre = create_image_association()

floor plan point selected: [1932, 1314]
floor plan point selected: [1556, 1332]
floor plan point selected: [1412, 524]
floor plan point selected: [2071, 772]
floor plan point selected: [1619, 764]
floor plan point selected: [1432, 257]
floor plan point selected: [942, 1355]
floor plan point selected: [546, 1336]
floor plan point selected: [760, 777]
floor plan point selected: [407, 803]


In [11]:
image_points_pre

array([[1932, 1314],
       [1556, 1332],
       [1412,  524],
       [2071,  772],
       [1619,  764],
       [1432,  257],
       [ 942, 1355],
       [ 546, 1336],
       [ 760,  777],
       [ 407,  803]])

In [45]:
# Initialize the camera matrix and distortion coefficients
camera_matrix = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]], dtype=np.float32)
dist_coeffs = np.zeros((5, 1), dtype=np.float32)

In [84]:
object_points = object_points_pre.reshape(14,3,1)
image_points = image_points_pre.reshape(14,2,1)
object_points = object_points.astype(np.float32)
image_points = image_points.astype(np.float32)

In [85]:
object_points_pre.reshape(14,3,1).shape, image_points.shape, camera_matrix.shape, dist_coeffs.shape

((14, 3, 1), (14, 2, 1), (3, 3), (5, 1))

In [86]:
object_points.dtype, image_points.dtype, camera_matrix.dtype, dist_coeffs.dtype

(dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'))

In [87]:

# Solve for the projection matrix
ret, rvec, tvec = cv2.solvePnP(object_points, image_points, camera_matrix, dist_coeffs)

In [90]:
ret, rvec, tvec

(True,
 array([[-1894.98729869],
        [-3437.7872221 ],
        [ 2115.60847867]]),
 array([[245634.77959094],
        [155092.01276606],
        [   652.5197138 ]]))

In [125]:

# Projection matrix (3x4)
projectionedPoints, _ = cv2.projectPoints(object_points, rvec, tvec, camera_matrix, dist_coeffs)

In [120]:
n = object_points.shape[0]
u = image_points[:,0]
v = image_points[:,1]
X = object_points[:,0]
Y = object_points[:,1]
Z = object_points[:,2]
A = np.zeros((2*n, 12))

for i in range(n):
    A[i*2]=[X[i], Y[i], Z[i], 1, 0, 0, 0, 0, -u[i]*X[i], -u[i]*Y[i], -u[i]*Z[i], -u[i]]
    A[i*2+1]=[0, 0, 0, 0, X[i], Y[i], Z[i], 1, -v[i]*X[i], -v[i]*Y[i], -v[i]*Z[i], -v[i]]

U, S, V = np.linalg.svd(A)
P = V[-1].reshape(3, 4)
	

/var/folders/8_/51vdws854m1218_v37ctyqj80000gn/T/ipykernel_39453/2638066416.py:10: DeprecationWarning:

setting an array element with a sequence. This was supported in some cases where the elements are arrays with a single element. For example `np.array([1, np.array([2])], dtype=int)`. In the future this will raise the same ValueError as `np.array([1, [2]], dtype=int)`.

/var/folders/8_/51vdws854m1218_v37ctyqj80000gn/T/ipykernel_39453/2638066416.py:11: DeprecationWarning:

setting an array element with a sequence. This was supported in some cases where the elements are arrays with a single element. For example `np.array([1, np.array([2])], dtype=int)`. In the future this will raise the same ValueError as `np.array([1, [2]], dtype=int)`.



In [127]:
P

array([[-1.04484533e-03,  1.28295639e-03,  4.46846177e-04,
        -9.31608840e-01],
       [ 3.14599779e-04, -4.51213353e-06,  1.08737493e-03,
        -3.63454687e-01],
       [ 3.75132794e-07,  9.55680445e-07,  3.28975413e-07,
        -1.20004307e-03]])

In [128]:
x = np.array([1925, 1258, 1]).dot(P)

In [133]:
x / x[0] * 1925

array([ 1.38184899e+00, -2.10756450e+00, -1.90577430e+00,  1.92500000e+03])

In [134]:
#It doesnt add up to the original image points